In [ ]:
import numpy as np
import pandas as pd

# Imports
import utils
import os
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
import seaborn as sns

from pathlib import Path
from adjustText import adjust_text

notebook_path = os.path.dirname(os.path.realpath("01_client_dropout_analysis.ipynb"))

In [ ]:
# Seaborn configuration
# Alternative font: Linux Libertine
sns.set_theme(context="paper", style="whitegrid", palette="colorblind", font="Times New Roman", font_scale=3)
sns.color_palette(palette="colorblind")
sns.set(rc={"figure.figsize": (14, 4)})
sns.set(font_scale=1.4)

In [ ]:
# Data download / load from disk
wandb_entity = "..."                                                                                                            # <-- Add you credentials for W&B here.
wandb_project = "..."                                                                                                           # <-- Add you credentials for W&B here.
wandb_run_filter_keywords = ["..."]                                                                                             # <-- Add you credentials for W&B here.
log_file_name = "wandb_dropout_dp_logs.csv"

if not Path(f"{notebook_path}/data/{log_file_name}").exists():
    # If logs have been downloaded already, they will be fetched from disk.
    df = utils.download_data_from_wandb(entity=wandb_entity, project=wandb_project, keywords=wandb_run_filter_keywords)
    df = utils.expand_experiment_name(df)
    utils.write_df_to_disk(df, filename=log_file_name)
else:
    df = pd.read_csv(f"{notebook_path}/data/{log_file_name}", index_col=0)
    df = utils.expand_experiment_name(df)
    utils.write_df_to_disk(df, filename=log_file_name)

In [ ]:
prox_mnist = df.loc[(df["dataset"] == "mnist") & (df["strategy"] == "fedprox") & (df["data_dist"] == "dirichlet") & (df["dp"] == "False")]
prox_pvt = prox_mnist.pivot_table(index=["dropout", "strategy"], columns=["dataset", "model"], values=["global_performance/accuracy"], aggfunc=[np.mean])
print(prox_pvt)

# RQ 3: Client Dropout analysis

In [ ]:
df3 = df.loc[df["dp"] == "False"] # We are not interested in DP experiments at this stage.
df3 = df3.loc[(df3["global_performance/f1_score"].notna()) & (df3["global_performance/f1_score"].notna())]
print(df["dp"].unique())

In [ ]:
# We report the global accuracy after the first, fifth, and last training round
df3 = df3[["dataset", "model", "strategy", "data_dist", "dropout", "epoch", "dp", "global_performance/accuracy", "global_performance/f1_score", "timing/evaluation_time", "_step", "id"]]

table_df3 = pd.DataFrame()
for dataset in ["blond", "mnist", "shakespeare"]:
    for model in ["cnn", "lstm", "resnet", "densenet"]:
        for strategy in ["fedavg", "qfedavg", "fedadam", "fedyogi", "fedadagrad", "fedprox"]:
            for dropout in [0, 1, 2, 5]:
                subset = df3.loc[(df3["dataset"] == dataset) & (df3["model"] == model) & (df3["strategy"] == strategy) & (df3["dropout"] == str(dropout)) & (df3["data_dist"] == "dirichlet") & (df3["dp"] == "False")]
                if len(subset) == 0:
                    continue

                experiments = subset["id"].unique().tolist()

                data = {"dataset": dataset, "model": model, "strategy": strategy, "dropout": dropout, "data_dist": "dirichlet"}
                exp_data = {"f1": [], "acc": []}
                for experiment in experiments:
                    exp = subset.loc[subset["id"] == experiment]
                    f1 = exp["global_performance/f1_score"].unique().tolist()
                    acc = exp["global_performance/accuracy"].unique().tolist()

                    while len(f1) != 11:
                        f1.append(f1[-1])

                    while len(acc) != 11:
                        acc.append(acc[-1])

                    exp_data["f1"].append(f1)
                    exp_data["acc"].append(acc)

                for rnd in range(0, 11):
                    round_vals = []
                    for exp in exp_data["f1"]:
                        round_vals.append(exp[rnd])

                    data[f"f1_round_{rnd}"] = np.nanmean(round_vals)
                    data[f"f1_round_{rnd}_stddev"] = np.nanstd(round_vals)

                    round_vals = []
                    for exp in exp_data["acc"]:
                        round_vals.append(exp[rnd])

                    data[f"acc_round_{rnd}"] = np.mean(round_vals)
                    data[f"acc_round_{rnd}_stddev"] = np.std(round_vals)

                table_df3 = pd.concat([table_df3, pd.DataFrame(data, index=[0])])
#
table_df3 = table_df3.round(decimals=4)


In [ ]:
# Here we investigate all non-IID datasets with Dirichlet alpha = 1.
table_df3["dropout"].replace([0, 1, 2, 5], [0, 0.1, 0.2, 0.5], inplace=True)
table_df3["strategy"].replace(["fedavg", "fedadam", "fedadagrad", "fedyogi", "qfedavg", "fedprox"], ["FedAvg", "FedAdam", "FedAdaGrad", "FedYogi", "qFedAvg", "FedProx"], inplace=True)
table_df3["dataset"].replace(["blond", "mnist", "shakespeare"], ["BLOND", "FEMNIST", "Shakespeare"], inplace=True)
table_df3["model"].replace(["cnn", "lstm", "resnet", "densenet"], ["CNN", "LSTM", "ResNet", "DenseNet"], inplace=True)

piv3 = table_df3.pivot_table(index=["dropout", "strategy"], columns=["dataset", "model"], values=["acc_round_10", "acc_round_10_stddev"])

for acc, dataset, model in [i for i in piv3.columns]:
    if "_stddev" not in acc:
        piv3[(acc, dataset, model)] = piv3[(acc, dataset, model)].round(decimals=2).astype(str)+ "±" + piv3[(f"{acc}_stddev", dataset, model)].round(decimals=2).astype(str)
    else:
        piv3.drop((acc, dataset, model), inplace=True, axis=1)

s = piv3.style.highlight_max(props='cellcolor:[HTML]{FFFF00}; color:{red}; itshape:; bfseries:;')
s.clear()
s.table_styles = []
s.format({
	("Numeric", "Integers"): '\${}',
	("Numeric", "Floats"): '{:.2f}',
    ("Non-Numeric", "Strings"): str.upper
})
print(s.to_latex(column_format="ll|rrrr|r|r", position="", position_float="centering", hrules=True, label="tab:speedup-comparison", caption="Speedup comparison across device types, datasets, and ML models.", multirow_align="t", multicol_align="r"))

In [ ]:
# We report the global accuracy after the first, fifth, and last training round
df4 = df3[["dataset", "model", "strategy", "data_dist", "dropout", "epoch", "dp", "global_performance/accuracy", "global_performance/f1_score", "timing/evaluation_time", "_step", "id"]]

table_df4 = pd.DataFrame()
for dataset in ["blond", "mnist", "shakespeare"]:
    for model in ["cnn", "lstm", "resnet", "densenet"]:
        for strategy in ["fedavg", "qfedavg", "fedadam", "fedyogi", "fedadagrad", "fedprox"]:
            for dropout in [0, 1, 2, 5]:
                subset = df4.loc[(df4["dataset"] == dataset) & (df4["model"] == model) & (df4["strategy"] == strategy) & (df4["dropout"] == str(dropout)) & (df4["data_dist"] == "iid") & (df4["dp"] == "False")]
                if len(subset) == 0:
                    continue

                experiments = subset["id"].unique().tolist()

                data = {"dataset": dataset, "model": model, "strategy": strategy, "dropout": dropout, "data_dist": "dirichlet"}
                exp_data = {"f1": [], "acc": []}
                for experiment in experiments:
                    exp = subset.loc[subset["id"] == experiment]
                    f1 = exp["global_performance/f1_score"].unique().tolist()
                    acc = exp["global_performance/accuracy"].unique().tolist()

                    while len(f1) != 11:
                        f1.append(f1[-1])

                    while len(acc) != 11:
                        acc.append(acc[-1])

                    exp_data["f1"].append(f1)
                    exp_data["acc"].append(acc)

                for rnd in range(0, 11):
                    round_vals = []
                    for exp in exp_data["f1"]:
                        round_vals.append(exp[rnd])

                    data[f"f1_round_{rnd}"] = np.nanmean(round_vals)
                    data[f"f1_round_{rnd}_stddev"] = np.nanstd(round_vals)

                    round_vals = []
                    for exp in exp_data["acc"]:
                        round_vals.append(exp[rnd])

                    data[f"acc_round_{rnd}"] = np.mean(round_vals)
                    data[f"acc_round_{rnd}_stddev"] = np.std(round_vals)

                table_df4 = pd.concat([table_df4, pd.DataFrame(data, index=[0])])
#
table_df4 = table_df4.round(decimals=4)

In [ ]:
# Here we investigate all non-IID datasets with Dirichlet alpha = 1.
table_df4["dropout"].replace([0, 1, 2, 5], [0, 0.1, 0.2, 0.5], inplace=True)
table_df4["strategy"].replace(["fedavg", "fedadam", "fedadagrad", "fedyogi", "qfedavg", "fedprox"], ["FedAvg", "FedAdam", "FedAdaGrad", "FedYogi", "qFedAvg", "FedProx"], inplace=True)
table_df4["dataset"].replace(["blond", "mnist", "shakespeare"], ["BLOND", "FEMNIST", "Shakespeare"], inplace=True)
table_df4["model"].replace(["cnn", "lstm", "resnet", "densenet"], ["CNN", "LSTM", "ResNet", "DenseNet"], inplace=True)

piv4 = table_df4.pivot_table(index=["dropout", "strategy"], columns=["dataset", "model"], values=["acc_round_10", "acc_round_10_stddev"])

for acc, dataset, model in [i for i in piv3.columns]:
    if "_stddev" not in acc:
        piv4[(acc, dataset, model)] = piv4[(acc, dataset, model)].round(decimals=2).astype(str)+ "±" + piv4[(f"{acc}_stddev", dataset, model)].round(decimals=2).astype(str)
    else:
        piv3.drop((acc, dataset, model), inplace=True, axis=1)

s = piv4.style.highlight_max(props='cellcolor:[HTML]{FFFF00}; color:{red}; itshape:; bfseries:;')
s.clear()
s.table_styles = []
s.format({
	("Numeric", "Integers"): '\${}',
	("Numeric", "Floats"): '{:.2f}',
    ("Non-Numeric", "Strings"): str.upper
})
print(s.to_latex(column_format="ll|rrrr|r|r", position="", position_float="centering", hrules=True, label="tab:speedup-comparison", caption="Speedup comparison across device types, datasets, and ML models.", multirow_align="t", multicol_align="r"))

# RQ 4: Differential Privacy Considerations

In [ ]:
df4 = df.loc[df["dp"] == "True"] # We are not interested in DP experiments at this stage.
df4_acc = df4.loc[(df4["global_performance/f1_score"].notna()) & (df4["global_performance/f1_score"].notna())]
df4_eps = df4.loc[df4["dp/epsilon"].notna()]

In [ ]:
# We report the global accuracy after the first, fifth, and last training round
df4_acc = df4_acc[["dataset", "model", "strategy", "data_dist", "dropout", "epoch", "global_performance/accuracy", "global_performance/f1_score", "id", "noise_multiplier"]].copy()
df4_eps = df4_eps[["dataset", "model", "strategy", "data_dist", "dropout", "id", "dp", "noise_multiplier", "dp/epsilon"]].copy()
# print(df4_eps)

table_df4 = pd.DataFrame()
for dataset in ["blond", "mnist", "shakespeare"]:
    for model in ["cnn", "lstm", "resnet", "densenet"]:
        for strategy in ["fedavg", "qfedavg", "fedadam", "fedyogi", "fedadagrad", "fedprox"]:
            for noise in [0, 0.3, 0.5, 1, 1.3, 1.5]:
                subset_acc = df4_acc.loc[(df4_acc["dataset"] == dataset) & (df4_acc["model"] == model) & (df4_acc["strategy"] == strategy) & (df4_acc["dropout"] == "0") & (df4_acc["noise_multiplier"] == str(noise))]
                subset_eps = df4_eps.loc[(df4_eps["dataset"] == dataset) & (df4_eps["model"] == model) & (df4_eps["strategy"] == strategy) & (df4_eps["noise_multiplier"] == str(noise))]

                if len(subset_acc) == 0:
                    continue

                experiments = subset_acc["id"].unique().tolist()
                data = {"dataset": dataset, "model": model, "strategy": strategy, "noise_multiplier": noise}
                exp_data = {"f1": [], "acc": [], "eps": []}
                for experiment in experiments:
                    exp_acc = subset_acc.loc[subset_acc["id"] == experiment]
                    exp_eps = subset_eps.loc[subset_eps["id"] == experiment]
                    f1 = exp_acc["global_performance/f1_score"].unique().tolist()
                    acc = exp_acc["global_performance/accuracy"].unique().tolist()
                    eps = exp_eps["dp/epsilon"].unique().tolist()

                    while len(f1) != 11:
                        f1.append(f1[-1])

                    while len(acc) != 11:
                        acc.append(acc[-1])

                    while len(eps) != 11:
                        eps.append(eps[-1])

                    exp_data["f1"].append(f1)
                    exp_data["acc"].append(acc)
                    exp_data["eps"].append(eps)

                for rnd in range(0, 11):
                    round_vals_f1 = []
                    for exp in exp_data["f1"]:
                        round_vals_f1.append(exp[rnd])

                    round_vals_acc = []
                    for exp in exp_data["acc"]:
                        round_vals_acc.append(exp[rnd])

                    round_vals_eps = []
                    for exp in exp_data["eps"]:
                        round_vals_eps.append(exp[rnd])

                    data["round"] = rnd
                    data["acc"] = np.mean(round_vals_acc)
                    data["acc_stddev"] = np.std(round_vals_acc)
                    data["f1"] = np.mean(round_vals_f1)
                    data["f1_stddev"] = np.std(round_vals_f1)
                    data["eps"] = np.mean(round_vals_eps)
                    data["eps_stddev"] = np.std(round_vals_eps)
                    table_df4 = pd.concat([table_df4, pd.DataFrame(data, index=[0])])


table_df4 = table_df4.round(decimals=4)

In [ ]:
# We report the global accuracy after the first, fifth, and last training round
df4_acc = df4_acc[["dataset", "model", "strategy", "data_dist", "dropout", "epoch", "global_performance/accuracy", "global_performance/f1_score", "id", "noise_multiplier"]].copy()
df4_eps = df4_eps[["dataset", "model", "strategy", "data_dist", "dropout", "id", "dp", "noise_multiplier", "dp/epsilon"]].copy()
# print(df4_eps)

table_df4 = pd.DataFrame()
for dataset in ["blond", "mnist", "shakespeare"]:
    for model in ["0cnn", "1lstm", "2resnet", "3densenet"]:
        for strategy in ["fedavg", "qfedavg", "fedadam", "fedyogi", "fedadagrad", "fedprox"]:
            for noise in [0, 0.3, 0.5, 1, 1.3, 1.5]:
                subset_acc = df4_acc.loc[(df4_acc["dataset"] == dataset) & (df4_acc["model"] == model) & (df4_acc["strategy"] == strategy) & (df4_acc["dropout"] == "0") & (df4_acc["noise_multiplier"] == str(noise))]
                subset_eps = df4_eps.loc[(df4_eps["dataset"] == dataset) & (df4_eps["model"] == model) & (df4_eps["strategy"] == strategy) & (df4_eps["noise_multiplier"] == str(noise))]

                if len(subset_acc) == 0:
                    continue

                experiments = subset_acc["id"].unique().tolist()
                data = {"dataset": dataset, "model": model, "strategy": strategy, "noise_multiplier": noise}
                exp_data = {"f1": [], "acc": [], "eps": []}
                for experiment in experiments:
                    exp_acc = subset_acc.loc[subset_acc["id"] == experiment]
                    exp_eps = subset_eps.loc[subset_eps["id"] == experiment]
                    f1 = exp_acc["global_performance/f1_score"].unique().tolist()
                    acc = exp_acc["global_performance/accuracy"].unique().tolist()
                    eps = exp_eps["dp/epsilon"].unique().tolist()

                    while len(f1) != 11:
                        f1.append(f1[-1])

                    while len(acc) != 11:
                        acc.append(acc[-1])

                    while len(eps) != 11:
                        eps.append(eps[-1])

                    exp_data["f1"].append(f1)
                    exp_data["acc"].append(acc)
                    exp_data["eps"].append(eps)

                for rnd in range(0, 11):
                    round_vals_f1 = []
                    for exp in exp_data["f1"]:
                        round_vals_f1.append(exp[rnd])

                    round_vals_acc = []
                    for exp in exp_data["acc"]:
                        round_vals_acc.append(exp[rnd])

                    round_vals_eps = []
                    for exp in exp_data["eps"]:
                        round_vals_eps.append(exp[rnd])

                    data["round"] = rnd
                    data["acc"] = np.mean(round_vals_acc)
                    data["acc_stddev"] = np.std(round_vals_acc)
                    data["f1"] = np.mean(round_vals_f1)
                    data["f1_stddev"] = np.std(round_vals_f1)
                    data["eps"] = np.mean(round_vals_eps)
                    data["eps_stddev"] = np.std(round_vals_eps)
                    table_df4 = pd.concat([table_df4, pd.DataFrame(data, index=[0])])


table_df4 = table_df4.round(decimals=4)

In [ ]:
sns.set(rc={"figure.figsize": (14, 4)})
sns.set(font_scale=1.4)

# See results for non-iid training without client dropouts.
add_metr = [
    {"dataset": "blond", "model": "cnn", "noise_multiplier": 0.0, "f1": 0.77},
    {"dataset": "blond", "model": "lstm", "noise_multiplier": 0.0, "f1": 0.77},
    {"dataset": "blond", "model": "resnet", "noise_multiplier": 0.0, "f1": 0.74},
    {"dataset": "blond", "model": "densenet", "noise_multiplier": 0.0, "f1": 0.74},
    {"dataset": "mnist", "model": "cnn", "noise_multiplier": 0.0, "f1": 0.71},
    {"dataset": "shakespeare", "model": "lstm", "noise_multiplier": 0.0, "f1": 0.53},
]

table_df4 = pd.concat([table_df4, pd.DataFrame(add_metr)])
for strategy in ["fedprox", "fedadam", "fedavg", "qfedavg", "fedyogi", "fedadagrad"]:
    tab4 = table_df4.loc[(table_df4["strategy"] == strategy)]
    pvt = tab4.pivot_table(index=["strategy", "dataset", "model"], columns="noise_multiplier", values=["f1"], aggfunc=np.mean)

    fig, ax1 = plt.subplots(1)
    pvt.plot(kind="bar", ax=ax1, xlabel="", ylabel="F1 Score", width=0.8)
    # ax1.set_xticklabels(["CNN", "LSTM", "ResNet", "DenseNet", "CNN", "LSTM"])
    ax1.axes.get_xaxis().set_visible(False)

    # Model axis
    ax2 = ax1.twiny()
    ax2.spines["bottom"].set_position(("axes", -0.05))
    ax2.tick_params('both', length=0, width=0, which='minor')
    ax2.tick_params('both', direction='in', which='major')
    ax2.xaxis.set_ticks_position("bottom")
    ax2.xaxis.set_label_position("bottom")

    ax2.set_xticks([0.166, 0.333, 0.50, 0.666, 0.833])
    ax2.xaxis.set_major_formatter(ticker.NullFormatter())
    ax2.xaxis.set_minor_locator(ticker.FixedLocator([0.08, 0.246, 0.416, 0.586, 0.753, 0.913]))
    ax2.xaxis.set_minor_formatter(ticker.FixedFormatter(["CNN", "LSTM", "ResNet", "DenseNet", "CNN", "LSTM"]))

    # Dataset axis
    ax3 = ax1.twiny()

    ax3.spines["bottom"].set_position(("axes", -0.13))
    ax3.tick_params('both', length=0, width=0, which='minor')
    ax3.tick_params('both', direction='in', which='major')
    ax3.xaxis.set_ticks_position("bottom")
    ax3.xaxis.set_label_position("bottom")

    ax3.set_xticks([0.666, 0.833])
    ax3.xaxis.set_major_formatter(ticker.NullFormatter())
    ax3.xaxis.set_minor_locator(ticker.FixedLocator([0.3333, 0.757, 0.913]))
    ax3.xaxis.set_minor_formatter(ticker.FixedFormatter(["BLOND", "FEMNIST", "Shakespeare"]))

    # Strategy axis
    ax3 = ax1.twiny()

    ax3.spines["bottom"].set_position(("axes", -0.20))
    ax3.tick_params('both', length=0, width=0, which='minor')
    ax3.tick_params('both', direction='in', which='major')
    ax3.xaxis.set_ticks_position("bottom")
    ax3.xaxis.set_label_position("bottom")

    ax3.set_xticks([])
    ax3.xaxis.set_major_formatter(ticker.NullFormatter())
    ax3.xaxis.set_minor_locator(ticker.FixedLocator([0.5]))
    ax3.xaxis.set_minor_formatter(ticker.FixedFormatter([f"Strategy: {strategy.upper()}"]))


    round_10_eps = table_df4[["acc", "eps"]][table_df4["round"] == 10]
    rects = ax1.patches


    labels = ["ε = ∞", "ε = ∞", "ε = ∞", "ε = ∞", "ε = ∞", "ε = ∞", ]
    labels += ["ε = 8.0", "ε = 8.0", "ε = 8.0", "ε = 8.0", "ε = 6.1", "ε = 6.6"]
    labels += ["ε = 2.3", "ε = 2.3", "ε = 2.3", "ε = 2.3", "ε = 2.1", "ε = 2.0"]
    labels += ["ε = 0.4", "ε = 0.4", "ε = 0.4", "ε = 0.4", "ε = 0.4", "ε = 0.4"]
    labels += ["ε = 0.2", "ε = 0.2", "ε = 0.2", "ε = 0.2", "ε = 0.3", "ε = 0.3"]
    labels += ["ε = 0.2", "ε = 0.2", "ε = 0.2", "ε = 0.2", "ε = 0.3", "ε = 0.3"]

    label_pos_cor = 0
    for idx, bar in enumerate(ax1.patches):
        # format(bar.get_height(), '.2f')
        ax1.annotate(labels[idx], (bar.get_x() + bar.get_width() / 2,bar.get_height() + 0.05), ha='center', va='center', size=12, xytext=(0, 8), textcoords='offset points', rotation=90)

    ax1.legend(["No DP", "z = 0.3", "z = 0.5", "z = 1.0", "z = 1.3", "z = 1.5"], ncols=3, fontsize=10)
    ax1.set_zorder(1)

    plt.ylim(0, 1.15)
    print(strategy)
    utils.write_figure_to_disk(plt=plt, file_name=f"dp_experiment_bar_chart_{strategy}", chapter_name="evaluations")